In [ ]:
import sys 
sys.path.append(rf"/nfs/home/genovese/thesis-wildfire-genovese/notebooks/")
from importlib import reload
import utils
reload(utils)
from utils import *
data_folder = "/nfs/home/genovese/thesis-wildfire-genovese/data/"

In [ ]:
extract_zipfile(os.path.join(data_folder, "Altitudini_comuni_DEM.zip"), os.path.join(data_folder, "raw/Altitudini_comuni_DEM/"))

In [ ]:
df = pd.read_csv(os.path.join(data_folder, "raw/Altitudini_comuni_DEM/Elab_Altimetrie_DEM.csv"), sep=';', encoding='latin1')
piemonte = df[df['COD_REG'] == 1].apply(lambda x: set_city_code(x, 'PRO_COM'), axis=1
                                        ).reset_index(drop=True
                                                      ).drop(['COD_REG', 'COD_PRO', 'NOME'], axis=1)
piemonte[['SUPERFICIE (in KMQ)', 'MEDIA', 'STD']] = piemonte[['SUPERFICIE (in KMQ)', 'MEDIA', 'STD']
                                                             ].apply(lambda x: x.str.replace(',', '.').astype(float))

gdf = gpd.read_file(os.path.join(data_folder, "clean_data/zone_altimetriche_piemonte/zone_altimetriche_piemonte.shp"))
result = pd.merge(gdf, piemonte.rename({'PRO_COM':'cod_istat'}, axis=1), on='cod_istat', how='inner'
         ).drop(['sezione', 'nome_sez', 'zalt_cod', 'nome_com'], axis=1)
save_clean_data(result, "zone_altimetriche_piemonte")

---

In [ ]:
dic = {'Pianura': 0,
       'Collina': 1,
       'Montagna': 2}
result['zalt'] = result['zalt_des'].map(dic)
result.drop(columns=['cod_istat', 'zalt_des'], inplace=True)
result.rename(columns={'zalt': 'zona_altimetrica'}, inplace=True)

dic = dict.fromkeys([col for col in result.columns if col not in ['geometry']])
for a in dic:
    dic[a] = 'ALTIMETRIA_'+a

result.rename(columns=dic, inplace=True)
result.to_file('/nfs/home/genovese/thesis-wildfire-genovese/database/altimetria.geojson', driver='GeoJSON', index=False)

---